In [1]:
#!pip install urdfpy
from xml.etree import ElementTree as ET
import numpy as np
#from urdfpy import URDF

In [ ]:
'''
robot = URDF.load('tu_robot.urdf')
for joint in robot.joints:
    print(joint.name, joint.origin, joint.axis)
'''

In [2]:
# Cargar el URDF como XML y analizar las juntas (joints)
tree = ET.parse('g1_29dof.urdf')
root = tree.getroot()

In [3]:
# Filtrar solo las articulaciones relevantes para el brazo (heurística por nombre)
relevant_keywords = ['shoulder', 'elbow', 'wrist']
arm_joints_full = []

for joint in root.findall('joint'):
    joint_name = joint.attrib.get('name', '')
    if any(k in joint_name.lower() for k in relevant_keywords):
        parent = joint.find('parent').attrib['link']
        child = joint.find('child').attrib['link']
        origin_tag = joint.find('origin')
        axis_tag = joint.find('axis')
        origin = origin_tag.attrib if origin_tag is not None else {}
        axis = axis_tag.attrib if axis_tag is not None else {}
        arm_joints_full.append({
            'name': joint_name,
            'parent': parent,
            'child': child,
            'origin': origin,
            'axis': axis
        })

#arm_joints_full

In [4]:
# Función auxiliar para convertir strings 'xyz' y 'rpy' en arrays numéricos
def parse_origin(origin_dict):
    xyz = np.fromstring(origin_dict.get('xyz', '0 0 0'), sep=' ')
    rpy = np.fromstring(origin_dict.get('rpy', '0 0 0'), sep=' ')
    return xyz, rpy

# Filtrar solo las articulaciones del brazo izquierdo en orden desde hombro hasta muñeca
left_arm_joint_names = [
    'left_shoulder_pitch_joint',
    'left_shoulder_roll_joint',
    'left_shoulder_yaw_joint',
    'left_elbow_joint',
    'left_wrist_roll_joint',
    'left_wrist_pitch_joint',
    'left_wrist_yaw_joint'
]

# Extraer los parámetros relevantes para las articulaciones del brazo izquierdo
left_arm_dh_data = []
for joint_name in left_arm_joint_names:
    joint = next(j for j in arm_joints_full if j['name'] == joint_name)
    xyz, rpy = parse_origin(joint['origin'])
    axis = np.fromstring(joint['axis'].get('xyz', '0 0 0'), sep=' ')
    
    left_arm_dh_data.append({
        'name': joint_name,
        'xyz': xyz,
        'rpy': rpy,
        'axis': axis
    })

left_arm_dh_data

[{'name': 'left_shoulder_pitch_joint',
  'xyz': array([0.0039563, 0.10022  , 0.23778  ]),
  'rpy': array([ 2.7931e-01,  5.4949e-05, -1.9159e-04]),
  'axis': array([0., 1., 0.])},
 {'name': 'left_shoulder_roll_joint',
  'xyz': array([ 0.      ,  0.038   , -0.013831]),
  'rpy': array([-0.27925,  0.     ,  0.     ]),
  'axis': array([1., 0., 0.])},
 {'name': 'left_shoulder_yaw_joint',
  'xyz': array([ 0.     ,  0.00624, -0.1032 ]),
  'rpy': array([0., 0., 0.]),
  'axis': array([0., 0., 1.])},
 {'name': 'left_elbow_joint',
  'xyz': array([ 0.015783,  0.      , -0.080518]),
  'rpy': array([0., 0., 0.]),
  'axis': array([0., 1., 0.])},
 {'name': 'left_wrist_roll_joint',
  'xyz': array([ 0.1       ,  0.00188791, -0.01      ]),
  'rpy': array([0., 0., 0.]),
  'axis': array([1., 0., 0.])},
 {'name': 'left_wrist_pitch_joint',
  'xyz': array([0.038, 0.   , 0.   ]),
  'rpy': array([0., 0., 0.]),
  'axis': array([0., 1., 0.])},
 {'name': 'left_wrist_yaw_joint',
  'xyz': array([0.046, 0.   , 0.   ])